In [1]:
%matplotlib inline

import sys
import os
import datetime as dt
from collections import OrderedDict, defaultdict

import pandas as pd
import numpy as np
import MySQLdb
from sklearn.metrics import silhouette_score, mean_squared_error, precision_recall_curve
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

### Class variable to connect with MySQL database

In [2]:
class dbConnect():
    '''
    Class to help with context management for 'with' statements.
    
    Example:
    c = dbConnect(host = 'localhost', user = 'root',
                  passwd = 'default', db = 'nba_stats')
    with c:
        df.to_sql('nbadotcom', c.con, flavor = 'mysql', dtype = dtype)
    '''
    def __init__(self, host, user, passwd, db):
        self.host = host
        self.user = user
        self.passwd = passwd
        self.db = db
    def __enter__(self):
        self.con = MySQLdb.connect(host = self.host, user = self.user,
                                   passwd = self.passwd, db = self.db)
        self.cur = self.con.cursor()
    def __exit__(self, type, value, traceback):
        self.cur.close()
        self.con.close()

In [3]:
# Example features that were explored: Team Statistics

columns = ['W_opt', 'L_opt', 'W_PCT_opt', 'PCT_FGA_2PT_opt', 'PCT_FGA_3PT_opt',
           'PCT_PTS_2PT_opt', 'PCT_PTS_2PT_MR_opt', 'PCT_PTS_3PT_opt',
           'PCT_PTS_FB_opt', 'PCT_PTS_FT_opt', 'PCT_PTS_OFF_TOV_opt',
           'PCT_PTS_PAINT_opt', 'PCT_AST_2PM_opt', 'PCT_UAST_2PM_opt',
           'PCT_AST_3PM_opt', 'PCT_UAST_3PM_opt', 'PCT_AST_FGM_opt',
           'PCT_UAST_FGM_opt', 'EFG_PCT_opt', 'FTA_RATE_opt', 'TM_TOV_PCT_opt',
           'OREB_PCT_opt', 'OPP_EFG_PCT_opt', 'OPP_FTA_RATE_opt',
           'OPP_TOV_PCT_opt', 'OPP_OREB_PCT_opt', 'PTS_OFF_TOV_opt',
           'PTS_2ND_CHANCE_opt', 'PTS_FB_opt', 'PTS_PAINT_opt',
           'OPP_PTS_OFF_TOV_opt', 'OPP_PTS_2ND_CHANCE_opt', 'OPP_PTS_FB_opt',
           'OPP_PTS_PAINT_opt', 'FGM_opt', 'FGA_opt', 'FG_PCT_opt', 'FG3M_opt',
           'FG3A_opt', 'FG3_PCT_opt', 'FTM_opt', 'FTA_opt', 'FT_PCT_opt',
           'OREB_opt', 'DREB_opt', 'REB_opt', 'AST_opt', 'TOV_opt', 'STL_opt',
           'BLK_opt', 'BLKA_opt', 'PF_opt', 'PFD_opt', 'PTS_opt',
           'PLUS_MINUS_opt', 'OFF_RATING_opt', 'DEF_RATING_opt',
           'NET_RATING_opt', 'AST_PCT_opt', 'AST_TO_opt', 'AST_RATIO_opt',
           'DREB_PCT_opt', 'REB_PCT_opt', 'TS_PCT_opt', 'PACE_opt', 'PIE_opt',
           'OPP_FGM_opt', 'OPP_FGA_opt', 'OPP_FG_PCT_opt', 'OPP_FG3M_opt',
           'OPP_FG3A_opt', 'OPP_FG3_PCT_opt', 'OPP_FTM_opt', 'OPP_FTA_opt',
           'OPP_FT_PCT_opt', 'OPP_OREB_opt', 'OPP_DREB_opt', 'OPP_REB_opt',
           'OPP_AST_opt', 'OPP_TOV_opt', 'OPP_STL_opt', 'OPP_BLK_opt',
           'OPP_BLKA_opt', 'OPP_PF_opt', 'OPP_PFD_opt', 'OPP_PTS_opt']

# Parameters for kMeans clustering
params_km = {'n_clusters': 5,
             'max_iter': 10000,
             'n_init': 10,
             'init': 'k-means++',
             'precompute_distances': 'auto',
             'tol': 0.0001,
             'n_jobs': 1,
             'verbose': 0}

### Class and function used for encoding categorical variables of interest

In [4]:
class PruneLabelEncoder(LabelEncoder):
    """
    Class variable that acts like LabelEncoder with the added functionality to 
    bin observations that appear with a low frequency defined by cutoff
    
    If the frequency of an encoded value is below the cutoff, it will bucket
    everything to the first value it encounters that is below the cutoff value
    """
    def __init___(self):
        super(PruneLabelEncoder, self).__init__()
    def fit(self, series, cutoff=10):
        self.cutoff = cutoff
        # Generate the transformation classes and the map for low output munging
        super(PruneLabelEncoder, self).fit(series)
        trans_series = super(PruneLabelEncoder, self).transform(series)
        self.val_count_map = defaultdict(int)
        for i in trans_series:
            self.val_count_map[i] += 1
        # Identify the first key with low frequency and use it for low freq vals
        for key, val in self.val_count_map.items():
            if val < self.cutoff:
                self.low_cnt_target = key
                break
    def transform(self, series):
        trans_series = super(PruneLabelEncoder, self).transform(series)
        # Transform all the low frequency keys into the low frequency target key
        for key, val in self.val_count_map.items():
            if val < self.cutoff:
                trans_series[trans_series==key] = self.low_cnt_target
        return trans_series

def encode(df, columns, TRANSFORM_CUTOFF):
    '''
    Takes in a dataframe, columns of interest, and a cutoff value for bucketing
    encoding values

    If the frequency of an encoded value is below the cutoff, it will bucket
    everything to the first value it encounters that is below the cutoff value
    '''
    temp = df.copy()

    # Checking if there are 2 or more unique values in each column
    for x in columns:
        if len(df[x].unique()) < 2:
            return 'Error: Fewer than 2 unique values in a column'

    for col in columns:
        le = PruneLabelEncoder()
        le.fit(df[col],TRANSFORM_CUTOFF)
        df[col] = le.transform(df[col])

    return df

## Functions used for quickly evaluating kMeans  Clustering

Used these functions to help decide which clusters to use for building my model

In [5]:
def km_cluster(df, params_km, n = 3, rounds = 100,
               columns = ['PLUS_MINUS_opt', 'PACE_opt']):
    '''
    Takes in dataframe (df), (n) number of clusters to explore,
    (rounds) number of rounds, and columns

    Returns a dataframe of the average and the standard deviation
    of value_counts()
    '''
    params_km['n_clusters'] = n
    dist = np.zeros(n)
    p = pd.DataFrame({'pred_0': dist})
    for x in range(rounds):
        est = KMeans(**params_km)
        X = df.as_matrix(columns).astype(float)
        est.fit(X)
        pred_km = est.predict(X)
        p['pred_'+str(x)] = np.array(pd.Series(pred_km).value_counts())
    results = pd.DataFrame({'mean': p.apply(np.mean, axis = 1),
                            'std': p.apply(np.std, axis = 1)})
    return results

def km_silhouette(df, params_km,
                  columns = ['PLUS_MINUS_opt', 'PACE_opt']):
    '''
    Takes in dataframe (df), (n) number of clusters to explore,
    (rounds) number of rounds, and columns

    Returns a dataframe of Silhouette Coefficients from kMeans
    '''
    
    cluster = np.arange(2,30)
    results = pd.DataFrame({'cluster': cluster})
    score = []
    for n in np.arange(2,30):
        params_km['n_clusters'] = n
        est = KMeans(**params_km)
        X = df.as_matrix(columns).astype(float)
        est.fit(X)
        labels = est.labels_        
        score += [silhouette_score(X, labels, metric='euclidean')]
    results['score'] = score
    return results

def plot_silhouette(df, params_km, columns):
    """
    Takes in dataframe (df), (n) number of clusters to explore,
    (rounds) number of rounds, and columns
    
    Returns a plot of the Silhouette Coefficient vs. the
    number of clusters from kMeans Clustering
    """
    plt.style.use('ggplot')
    fig, ax = plt.subplots(figsize = (10, 8))
    
    km_df = km_silhouette(df, params_km, columns)
    
    ax.plot(km_df['cluster'],km_df['score'], color = 'black',
           label = 'Silhouette Coefficient',
           lw = 3, alpha = 0.6, ls = 'dashed')
    ax.set_xlabel('Number of kMeans Clusters',fontsize = 14)
    ax.set_ylabel('Silhouette Coefficient', fontsize = 14)
    ax.set_title('kMeans Analysis of Silhouette Coefficient', fontsize = 20)    

def add_cluster(df, columns, params_km):
    '''
    Takes in: dataframe (df), columns to analyze, kMeans parameters (params_km)

    Returns: Dataframe with a new column for the cluster (to help categorize)
    '''
    est = KMeans(**params_km)
    X = df.as_matrix(columns).astype(float)
    est.fit(X)
    pred_km = est.predict(X)
    df['cluster'] = pred_km
    return df

### Add new features to large dataframe

In [6]:
def make_df(params_km = params_km, columns = columns):
    '''
    Takes in kMeans parameters and columns to perform kMeans

    Returns a merged dataframe with a new 'cluster' column (from kMeans)
    '''
    
    # Grab opponent stats from MySQL database
    db = 'nba_stats'
    temp = dbConnect(host = 'localhost', user = 'root',
                     passwd = 'default', db = db)
    with temp:
        sql = 'SELECT * FROM nba_opponent_allstats_2015'
        nba_df = pd.read_sql_query(sql, con = temp.con, index_col = 'index')

    est = KMeans(**params_km)
    X = nba_df.as_matrix(columns).astype(float)
    est.fit(X)
    pred_km = est.predict(X)
    nba_df['cluster'] = pred_km
    
    # Merge full dataframe with new cluster feature
    bigdf = pd.read_csv('../csv_data/nba_15season_all_150928.csv')
    d = nba_df[['opponent','cluster']].copy()
    bigdf = pd.merge(bigdf, d, how = 'left', on = 'opponent')
    
    # Add redundant column (using minutes_pm column to subset later)
    # the 'minutes' column will be converted to avg minutes based on cluster
    bigdf['minutes_pm'] = bigdf['minutes']
    columns = list(bigdf.columns)
    columns.remove('points')
    columns.append('points')
    bigdf = bigdf[columns]

    return bigdf

def cluster_features(bigdf):
    '''
    Takes in dataframe from make_df()

    Returns a new dataframe that averages bref stats over each cluster for the
    months not in March or April.
    '''
    # Columns for calculate summary stats over for each cluster
    brefcol = ['minutes', 'num_poss', 'opp_poss', 'pace_bref',
                'fg_pm', 'fga_pm', 'fg_percent', 'TP_pm',
                'TPA_pm', 'TP_percent', 'eFG', 'FT_pm',
                'FTA_pm', 'FT_percent', 'cluster']

    df = bigdf.copy()
    df_std = bigdf.copy()
    df_max = bigdf.copy()
    df_min = bigdf.copy()

    for lineup in df['lineup'].unique():
        # t_df is a dataframe of all information not associated with March or April
        t_df = df[(df['lineup'] == lineup) &
                  np.logical_not(df['month'].isin([3,4]))]
        for x in t_df['cluster'].unique():
            # Making mean columns (aggregate cluster by mean)
            temp_df = pd.DataFrame(t_df.loc[t_df['cluster'] == x,
                                   brefcol[:-1]].apply(np.mean, axis = 0))
            temp_df = temp_df.T
            temp_df = temp_df.as_matrix(brefcol[:-1])
            df.loc[(df['lineup'] == lineup) & (df['cluster'] == x),
                    brefcol[:-1]] = temp_df

            # Making stdev columns
            temp_std = pd.DataFrame(t_df.loc[t_df['cluster'] == x,
                                   brefcol[:-1]].apply(np.std, axis = 0))
            temp_std = temp_std.T
            temp_std = temp_std.as_matrix(brefcol[:-1])
            df_std.loc[(df_std['lineup'] == lineup) & (df_std['cluster'] == x),
                    brefcol[:-1]] = temp_std

            # Making max columns
            temp_max = pd.DataFrame(t_df.loc[t_df['cluster'] == x,
                                   brefcol[:-1]].apply(np.max, axis = 0))
            temp_max = temp_max.T
            temp_max = temp_max.as_matrix(brefcol[:-1])
            df_max.loc[(df_max['lineup'] == lineup) & (df_max['cluster'] == x),
                    brefcol[:-1]] = temp_max

            # Making min columns
            temp_min = pd.DataFrame(t_df.loc[t_df['cluster'] == x,
                                   brefcol[:-1]].apply(np.min, axis = 0))
            temp_min = temp_min.T
            temp_min = temp_min.as_matrix(brefcol[:-1])
            df_min.loc[(df_min['lineup'] == lineup) & (df_min['cluster'] == x),
                    brefcol[:-1]] = temp_min
    
    # Renaming columns and dropping the cluster column (the last column)
    df = df.drop('cluster', axis = 1)
    
    df_std = df_std[brefcol[:-1]]
    df_std.columns = [col + '_std' for col in df_std.columns]

    df_max = df_max[brefcol[:-1]]
    df_max.columns = [col + '_max' for col in df_max.columns]

    df_min = df_min[brefcol[:-1]]
    df_min.columns = [col + '_min' for col in df_min.columns]

    print 'df shape:', df.shape
    print 'df_std shape:', df_std.shape
    print 'df_max shape:', df_max.shape
    print 'df_min shape:', df_min.shape

    # Merge dataframes together on the index
    df = df.reset_index()
    df = pd.merge(df, df_std.reset_index(), how = 'left', on = 'index')
    df = pd.merge(df, df_max.reset_index(), how = 'left', on = 'index')
    df = pd.merge(df, df_min.reset_index(), how = 'left', on = 'index')
    df = df.drop('index', axis = 1)
    
    # Placing the points column as the last column
    columns = list(df.columns)   
    columns.remove('points')
    columns.append('points')
    df = df[columns]
    return df

In [7]:
# Example Usage          
        
# Grab opponent stats from MySQL database
db = 'nba_stats'
temp = dbConnect(host = 'localhost', user = 'root',
                 passwd = 'default', db = db)
with temp:
    sql = 'SELECT * FROM nba_opponent_allstats_2015'
    nba_df = pd.read_sql_query(sql, con = temp.con, index_col = 'index')

df = encode(df = nba_df, columns = ['opponent'], TRANSFORM_CUTOFF = 1)
# Columns from beginning of Notebook
print km_cluster(df, params_km, n = 3, rounds = 100, columns = columns)

   mean  std
0    12    0
1    11    0
2     7    0
